In [1]:
window= 60
embed_size = 128
maxlen = 150

In [2]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import os

In [3]:
train_root_path ='./data/train_preliminary/'
train_ad_path = os.path.join(train_root_path,'ad.csv')
train_click_path = os.path.join(train_root_path,'click_log.csv')
train_user_path = os.path.join(train_root_path,'user.csv')

In [4]:
semi_train_root_path ='./data/train_semi_final/'
semi_train_ad_path = os.path.join(semi_train_root_path,'ad.csv')
semi_train_click_path = os.path.join(semi_train_root_path,'click_log.csv')
semi_train_user_path = os.path.join(semi_train_root_path,'user.csv')

In [5]:
test_root_path = './data/test/'
test_ad_path = os.path.join(test_root_path,'ad.csv')
test_click_path = os.path.join(test_root_path,'click_log.csv')

# 合并 初赛和半决赛数据

In [6]:
df_click_train = pd.read_csv(train_click_path,dtype= {'creative_id':str,'click_times':str,'time':str})
df_semi_click_train = pd.read_csv(semi_train_click_path,dtype= {'creative_id':str,'click_times':str,'time':str})
df_click_test = pd.read_csv(test_click_path,dtype= {'creative_id':str,'click_times':str,'time':str})
df_click_all =  pd.concat([df_click_train,df_semi_click_train,df_click_test], axis=0)
df_click_all.drop_duplicates(inplace=True)

In [7]:
df_ad_train = pd.read_csv(train_ad_path,na_values='\\N',dtype= {'creative_id':str,'ad_id': str, 'product_id': str, 'product_category': str,'advertiser_id': str,'industry': str} )
df_semi_ad_train = pd.read_csv(semi_train_ad_path,na_values='\\N',dtype= {'creative_id':str,'ad_id': str, 'product_id': str, 'product_category': str,'advertiser_id': str,'industry': str} )
df_ad_test = pd.read_csv(test_ad_path,na_values='\\N',dtype= {'creative_id':str,'ad_id': str, 'product_id': str, 'product_category': str,'advertiser_id': str,'industry': str} )
df_ad_all =  pd.concat([df_ad_train,df_semi_ad_train,df_ad_test], axis=0)
df_ad_all.drop_duplicates(inplace=True)

In [8]:
df_user = pd.read_csv(train_user_path)
df_semi_user = pd.read_csv(semi_train_user_path)
df_user_all =  pd.concat([df_user,df_semi_user], axis=0)
df_user_all.drop_duplicates(inplace=True)

In [9]:
df_semi_user.shape

(2100000, 3)

In [10]:
df_doc = df_click_all.merge(df_ad_all,on=['creative_id'],how='inner')


In [11]:
ad_docs_withuser = df_doc.merge(df_user_all,on=['user_id'],how='left')

In [12]:
ad_docs_withuser.head(3)

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry,age,gender
0,9,30920,567330,1,504423,30673,3,32638,319,2.0,1.0
1,15,320815,567330,1,504423,30673,3,32638,319,3.0,1.0
2,11,355089,567330,1,504423,30673,3,32638,319,1.0,1.0


In [13]:
# del df_click_train
# del df_semi_click_train
# del df_click_test
# del df_ad_train
# del df_semi_ad_train
# del df_ad_test
# del df_doc
# import gc
# gc.collect()

In [14]:
# ad_docs_withuser = None

In [16]:
ad_docs_withuser.to_parquet('./data/cache/ad_docs_withuser.parquet',index=False)

# 点击序列 生成

In [17]:
# ad_docs_withuser = pd.read_parquet('./data/cache/ad_docs_withuser.parquet')

In [18]:
ad_docs_withuser = ad_docs_withuser.sort_values('time', ascending=True)

In [19]:
ad_docs_withuser.fillna('un',inplace=True)

In [20]:
from gensim.models.callbacks import CallbackAny2Vec
class epoch_logger(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0
        self.loss_to_be_subed = 0
        
    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))
        
    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        loss_now = loss - self.loss_to_be_subed
        self.loss_to_be_subed = loss
        print('Loss after epoch {}: {}'.format(self.epoch, loss_now))
        self.epoch += 1

In [21]:
aggcol = ['creative_id','ad_id','product_id','product_category','advertiser_id','industry']


In [22]:
func = lambda x: ' '.join(x.tolist())
agg_funcs ={}
for col in aggcol:
    agg_funcs[col] = func
doc = ad_docs_withuser.groupby(['user_id']).agg(agg_funcs)
doc = doc.reset_index()

In [23]:
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

new_df ={}
new_df['user_id'] = doc['user_id']
word_index ={}
tokenizer = None
for col in tqdm(aggcol):
    tokenizer = Tokenizer(lower=False, char_level=False, split=' ')
    tokenizer.fit_on_texts(doc[col])
    train_x = tokenizer.texts_to_sequences(doc[col])
    train_x = pad_sequences(train_x, maxlen=maxlen, value=0,padding='post')
    new_df[col] = list(train_x)
    word_index[col] =tokenizer.word_index
df_final = pd.DataFrame(new_df)
df_final.to_parquet('./data/cache/final_padding.parquet_8input',index=False)

100%|██████████| 6/6 [29:48<00:00, 298.12s/it]


In [24]:
import pickle
f = open("./data/cache/word_index.pkl","wb")
pickle.dump(word_index,f)
f.close()

In [25]:
func = lambda x: list(x)
agg_funcs={}
for col in aggcol:
    agg_funcs[col] = func
doc = ad_docs_withuser.groupby(['user_id']).agg(agg_funcs)
doc = doc.reset_index()

In [26]:
#product_docs = df_click_all.groupby(['user_id'])['product_id'].apply(list).reset_index(name='product_list')
#agg_col = ['ad_id','product_id','product_category','advertiser_id','industry','click_times','time']
for col in aggcol:
    print(f'Start {col} embeding:')
    w2v_model = Word2Vec(doc[col], sg=1,window=window,min_count=1,size=embed_size,iter=7,workers=32, compute_loss=True,callbacks=[epoch_logger()])
    w2v_model.wv.save(f"./model/word2vec_{col}_128.model")

Start creative_id embeding:
Epoch #0 start
Loss after epoch 0: 73031072.0
Epoch #1 start
Loss after epoch 1: 15308392.0
Epoch #2 start
Loss after epoch 2: 13864480.0
Epoch #3 start
Loss after epoch 3: 12713784.0
Epoch #4 start
Loss after epoch 4: 11551184.0
Epoch #5 start
Loss after epoch 5: 7748816.0
Epoch #6 start
Loss after epoch 6: 0.0
Start ad_id embeding:
Epoch #0 start
Loss after epoch 0: 73136328.0
Epoch #1 start
Loss after epoch 1: 15349272.0
Epoch #2 start
Loss after epoch 2: 14047016.0
Epoch #3 start
Loss after epoch 3: 12917936.0
Epoch #4 start
Loss after epoch 4: 11545696.0
Epoch #5 start
Loss after epoch 5: 7221480.0
Epoch #6 start
Loss after epoch 6: 0.0
Start product_id embeding:
Epoch #0 start
Loss after epoch 0: 45202396.0
Epoch #1 start
Loss after epoch 1: 23451588.0
Epoch #2 start
Loss after epoch 2: 4335216.0
Epoch #3 start
Loss after epoch 3: 3912224.0
Epoch #4 start
Loss after epoch 4: 3425240.0
Epoch #5 start
Loss after epoch 5: 2872072.0
Epoch #6 start
Loss aft

In [27]:
#advertiser_id 没完成
import gc
gc.collect()

0

# click_time 生成mask

In [28]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [29]:
ad_docs_withuser['click_times'] = ad_docs_withuser['click_times'].astype(int)
ad_docs_withuser = ad_docs_withuser.groupby(['user_id'])['click_times'].apply(list).reset_index()

In [30]:
train_click = pad_sequences(ad_docs_withuser['click_times'], maxlen=maxlen, value=0,padding='post')

In [31]:
train_click

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [2, 1, 1, ..., 0, 0, 0]], dtype=int32)

In [32]:
click_lenth = ad_docs_withuser['click_times'].apply(lambda x: len(x))

In [33]:
new_df ={}
new_df['user_id'] = ad_docs_withuser['user_id']
new_df['click_times'] =  list(train_click)
new_df['click_times_length'] = click_lenth

In [34]:
df_final = pd.DataFrame(new_df)
df_final.to_parquet('./data/cache/click_times_mask.parquet',index=False)

# deepwalk 生成其他序列

In [ ]:
#Deepwalk算法，
def generate_deep(log,f1,f2,L):
    gc.collect()
    print("deepwalk:",f1,f2)
    #构建图
    dic={}
    for item in log[[f1,f2]].values:
        try:
            str(item[1])
            str(item[0])
        except:
            continue
        item_key = 'item_'+str(item[1])
        user_key = 'user_'+str(item[0])
        if item_key in dic:
            dic[item_key].add(user_key)
        else:
            dic[item_key] = set([user_key])
        if user_key in dic:
            dic[user_key].add(item_key)
        else:
            dic[user_key] = set([item_key])
    dic_cont={}
    for key in dic:
        dic[key]=list(dic[key])
        dic_cont[key]=len(dic[key])
    print("creating")     
    #构建路径
    path_length=30        
    sentences=[]
    length=[]
    for key in dic:
        sentence=[key]
        while len(sentence)!=path_length:
            key=dic[sentence[-1]][random.randint(0,dic_cont[sentence[-1]]-1)]
            if len(sentence)>=2 and key == sentence[-2]:
                break
            else:
                sentence.append(key)
        sentence = [key[5:] for key in sentence if key.startswith('item_')]
        sentences.append(sentence)
        length.append(len(sentence))
    print(np.mean(length))
    print(len(sentences))
    #训练Deepwalk模型
    print('training...')
    np.random.shuffle(sentences)
    model = Word2Vec(sentences, size=L, window=10,min_count=1,sg=1, iter=5,compute_loss=True,callbacks=[epoch_logger()])
    model.wv.save(f"./model/deepwalk/word2vec_{f2}_{str(L)}.model")

In [ ]:
for agg in aggcol:
    generate_deep(df_click_all,'user_id',agg,128)